<a href="https://colab.research.google.com/github/vladimiralencar/DeepLearning-LANA/blob/master/CNN/CNN02.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CNN Para Reconhecimento de Imagens com CIFAR-10

## Criando um ambiente virtual no Google Colab

In [0]:
!pip install virtualenv
!virtualenv tf
!source tf/bin/activate
!pip uninstall --y tensorflow 
!pip install tensorflow==1.7 prettytensor==0.7.4

In [0]:
!source tf/bin/activate
import tensorflow as tf
tf.__version__

## Carregando arquivos

In [0]:
!wget https://raw.githubusercontent.com/vladimiralencar/DeepLearning-LANA/master/CNN/images/network_flowchart.png
!mkdir images
!cp *.png images

In [0]:
!wget https://raw.githubusercontent.com/vladimiralencar/DeepLearning-LANA/master/CNN/cache.py
!wget https://raw.githubusercontent.com/vladimiralencar/DeepLearning-LANA/master/CNN/cifar10.py
!wget https://raw.githubusercontent.com/vladimiralencar/DeepLearning-LANA/master/CNN/dataset.py
!wget https://raw.githubusercontent.com/vladimiralencar/DeepLearning-LANA/master/CNN/download.py
!ls

In [0]:
#from google.colab import files
#files.upload()
#!mkdir images
#!cp *.png images

## Introdução

Veremos agora como construir uma Rede Neural Convolucional para classificar imagens no conjunto de dados CIFAR-10. 

https://www.cs.toronto.edu/~kriz/cifar.html

## Flowchart

O gráfico a seguir mostra aproximadamente como o fluxo de dados da Rede Neural Convolucional é implementado abaixo. Primeiro, a rede possui uma camada de pré-processamento que distorce as imagens de entrada de modo a inflar artificialmente o conjunto de treinamento. Em seguida, a rede possui duas camadas convolucionais, duas camadas totalmente conectadas e, finalmente, uma camada de classificação softmax. Os pesos e saídas das camadas convolucionais são mostrados com mais detalhes nos gráficos maiores abaixo.

In [0]:
from IPython.display import Image
Image('images/network_flowchart.png')

In [0]:
# Verifique se GPU tem memória disponível
!nvidia-smi

## Imports

In [0]:
%matplotlib inline
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
from sklearn.metrics import confusion_matrix
import time
from datetime import timedelta
import math
import os

# O PrettyTensor simplifica a construção de Redes Neurais com TensorFlow
# https://github.com/google/prettytensor
import prettytensor as pt

Esse modelo foi desenvolvido usando a versão Python 3.6 (Anaconda) e TensorFlow:

In [0]:
tf.__version__

Versão do PrettyTensor:

In [0]:
pt.__version__

## Carregando os Dados

In [0]:
import cifar10

In [0]:
# O conjunto de dados CIFAR-10 é de cerca de 163 MB e será baixado automaticamente caso não seja encontrado.
cifar10.maybe_download_and_extract()

In [0]:
class_names = cifar10.load_class_names()
class_names

In [0]:
images_train, cls_train, labels_train = cifar10.load_training_data()

In [0]:
images_test, cls_test, labels_test = cifar10.load_test_data()

In [0]:
print("- Dataset de Treino:\t\t{}".format(len(images_train)))
print("- Dataset de Teste:\t\t{}".format(len(images_test)))

In [0]:
from cifar10 import img_size, num_channels, num_classes

In [0]:
# As imagens são 32x32, mas iremos reduzir o tamanho para 24x24 a fim de dminuir o tempo de treinamento
img_size_cropped = 24

### Função Para Plot de Imagens

In [0]:
def plot_images(images, cls_true, cls_pred=None, smooth=True):

    assert len(images) == len(cls_true) == 9

    fig, axes = plt.subplots(3, 3)

    if cls_pred is None:
        hspace = 0.3
    else:
        hspace = 0.6
    fig.subplots_adjust(hspace=hspace, wspace=0.3)

    for i, ax in enumerate(axes.flat):
        if smooth:
            interpolation = 'spline16'
        else:
            interpolation = 'nearest'

        ax.imshow(images[i, :, :, :], interpolation=interpolation)
            
        cls_true_name = class_names[cls_true[i]]

        if cls_pred is None:
            xlabel = "True: {0}".format(cls_true_name)
        else:
            cls_pred_name = class_names[cls_pred[i]]

            xlabel = "Label Original: {0}\nPrevisão: {1}".format(cls_true_name, cls_pred_name)

        ax.set_xlabel(xlabel)
        
        ax.set_xticks([])
        ax.set_yticks([])
    
    plt.show()

### Plot de Algumas Imagens

In [0]:
images = images_test[0:9]

cls_true = cls_test[0:9]

plot_images(images=images, cls_true=cls_true, smooth=False)

In [0]:
# As imagens pixeladas acima são o que a rede neural receberá como entrada. 
# As imagens podem ser um pouco mais fáceis para o olho humano reconhecer se "alisamos" os pixels.
plot_images(images=images, cls_true=cls_true, smooth=True)

## Criando o Grafo no TensorFlow 

### Placeholders

In [0]:
x = tf.placeholder(tf.float32, shape=[None, img_size, img_size, num_channels], name='x')

In [0]:
y_true = tf.placeholder(tf.float32, shape=[None, num_classes], name='y_true')

In [0]:
y_true_cls = tf.argmax(y_true, dimension=1)

### Função Para Pré-Processamento dos Dados

In [0]:
def pre_process_image(image, training):
 
    if training:
        image = tf.random_crop(image, size=[img_size_cropped, img_size_cropped, num_channels])

        image = tf.image.random_flip_left_right(image)
        
        image = tf.image.random_hue(image, max_delta=0.05)
        image = tf.image.random_contrast(image, lower=0.3, upper=1.0)
        image = tf.image.random_brightness(image, max_delta=0.2)
        image = tf.image.random_saturation(image, lower=0.0, upper=2.0)
        image = tf.minimum(image, 1.0)
        image = tf.maximum(image, 0.0)
    else:
        image = tf.image.resize_image_with_crop_or_pad(image,
                                                       target_height=img_size_cropped,
                                                       target_width=img_size_cropped)

    return image

A função acima é chamada para cada imagem no lote de entrada usando a seguinte função.

In [0]:
def pre_process(images, training):
    images = tf.map_fn(lambda image: pre_process_image(image, training), images)

    return images

In [0]:
distorted_images = pre_process(images=x, training=True)

### Função Para Definir a Rede Neural Convolucional

In [0]:
def main_network(images, training):
    x_pretty = pt.wrap(images)

    if training:
        phase = pt.Phase.train
    else:
        phase = pt.Phase.infer

    with pt.defaults_scope(activation_fn = tf.nn.relu, phase = phase):
        y_pred, loss = x_pretty.\
            conv2d(kernel = 5, depth = 64, name = 'layer_conv1', batch_normalize = True).\
            max_pool(kernel = 2, stride = 2).\
            conv2d(kernel = 5, depth = 64, name = 'layer_conv2').\
            max_pool(kernel = 2, stride = 2).\
            flatten().\
            fully_connected(size = 256, name = 'layer_fc1').\
            fully_connected(size = 128, name = 'layer_fc2').\
            softmax_classifier(num_classes=num_classes, labels=y_true)

    return y_pred, loss

### Função Para Criar a Rede Neural Convolucional

In [0]:
def create_network(training):
    with tf.variable_scope('network', reuse=not training):
        images = x

        images = pre_process(images=images, training=training)

        y_pred, loss = main_network(images=images, training=training)

    return y_pred, loss

### Treinamento

Primeiro criamos uma variável TensorFlow que acompanha o número de iterações de otimização realizadas até o momento. 

Note que `trainable = False` o que significa que o TensorFlow não tentará otimizar esta variável.

In [0]:
global_step = tf.Variable(initial_value = 0, name = 'global_step', trainable = False)

In [0]:
_, loss = create_network(training=True)

In [0]:
optimizer = tf.train.AdamOptimizer(learning_rate=1e-4).minimize(loss, global_step=global_step)

### Teste / Inferência

In [0]:
y_pred, _ = create_network(training=False)

In [0]:
y_pred_cls = tf.argmax(y_pred, dimension=1)

In [0]:
correct_prediction = tf.equal(y_pred_cls, y_true_cls)

In [0]:
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

### Salvando o Modelo

In [0]:
saver = tf.train.Saver()

### Obtendo os Pesos

Mais adiante, queremos plotar os pesos da rede neural. Quando a rede é construída usando o Pretty Tensor, todas as variáveis das camadas são criadas indiretamente pelo Pretty Tensor. Portanto, devemos recuperar as variáveis do TensorFlow.

Utilizamos os nomes `layer_conv1` e` layer_conv2` para as duas camadas convolucionais. Estes também são chamados escopos variáveis. Pretty Tensor automaticamente dá nomes às variáveis que cria para cada camada, para que possamos recuperar os pesos para uma camada usando o nome do escopo da camada e o nome da variável.

A implementação é um pouco incompreensível porque precisamos usar a função TensorFlow `get_variable ()` que foi projetada para outra finalidade; Criando uma nova variável ou reutilizando uma variável existente. O mais fácil é fazer a seguinte função auxiliar.

In [0]:
def get_weights_variable(layer_name):
    
    with tf.variable_scope("network/" + layer_name, reuse=True):
        variable = tf.get_variable('weights')

    return variable

In [0]:
weights_conv1 = get_weights_variable(layer_name='layer_conv1')
weights_conv2 = get_weights_variable(layer_name='layer_conv2')

### Obtendo os Outputs das Camadas

In [0]:
def get_layer_output(layer_name):
    tensor_name = "network/" + layer_name + "/Relu:0"

    tensor = tf.get_default_graph().get_tensor_by_name(tensor_name)

    return tensor

In [0]:
output_conv1 = get_layer_output(layer_name='layer_conv1')
output_conv2 = get_layer_output(layer_name='layer_conv2')

## TensorFlow Run

### Sessão TensorFlow

In [0]:
session = tf.Session()

### Restaurar ou inicializar variáveis

Treinar esta rede neural pode demorar muito tempo, especialmente se você não possui GPU. Portanto, incluímos pontos de controle durante o treinamento para que possamos continuar treinando em outro momento (por exemplo durante a noite), e também para realizar análises depois sem ter que treinar a rede neural sempre que quisermos usá-la.

Se você quiser reiniciar o treinamento da rede neural, você deve primeiro excluir os pontos de verificação.

Este é o diretório usado para os pontos de verificação.

In [0]:
save_dir = 'checkpoints/'

In [0]:
if not os.path.exists(save_dir):
    os.makedirs(save_dir)

In [0]:
save_path = os.path.join(save_dir, 'cifar10_cnn')

In [0]:
try:
    print("Tentando restaurar o último ponto de controle ...")

    last_chk_path = tf.train.latest_checkpoint(checkpoint_dir=save_dir)

    saver.restore(session, save_path=last_chk_path)

    print("Ponto de verificação restaurado de:", last_chk_path)
except:
    print("Não foi possível restaurar o ponto de verificação. Inicializando variáveis em vez disso.")
    session.run(tf.global_variables_initializer())

### Função Para Gerar um training-batch

Há 50.000 imagens no conjunto de treinamento. Demora muito tempo calcular o gradiente do modelo usando todas essas imagens. Portanto, usamos apenas um pequeno lote de imagens em cada iteração do otimizador.

Se o seu computador falhar ou se tornar muito lento porque você está sem memória RAM, então você pode tentar diminuir esse número, mas talvez seja necessário executar mais iterações de otimização.

In [0]:
train_batch_size = 64

In [0]:
def random_batch():
    num_images = len(images_train)

    idx = np.random.choice(num_images, size=train_batch_size, replace=False)

    x_batch = images_train[idx, :, :, :]
    y_batch = labels_train[idx, :]

    return x_batch, y_batch

### Função Para Otimização

Esta função executa uma série de iterações de otimização para melhorar gradualmente as variáveis das camadas da rede. Em cada iteração, um novo lote de dados é selecionado a partir do conjunto de treinamento e, em seguida, TensorFlow executa o otimizador usando essas amostras de treinamento. O progresso é impresso a cada 100 iterações. Um ponto de verificação é salvo a cada 1000 iterações e também após a última iteração.

In [0]:
def optimize(num_iterations):
    start_time = time.time()

    for i in range(num_iterations):
        x_batch, y_true_batch = random_batch()

        feed_dict_train = {x: x_batch, y_true: y_true_batch}

        i_global, _ = session.run([global_step, optimizer], feed_dict=feed_dict_train)

        if (i_global % 100 == 0) or (i == num_iterations - 1):
            batch_acc = session.run(accuracy, feed_dict=feed_dict_train)

            # Print status.
            msg = "Global Step: {0:>6}, Acurácia no Training Batch: {1:>6.1%}"
            print(msg.format(i_global, batch_acc))

        if (i_global % 1000 == 0) or (i == num_iterations - 1):
            saver.save(session, save_path=save_path, global_step=global_step)

            print("Checkpoint Salvo.")

    end_time = time.time()

    time_dif = end_time - start_time

    print("Tempo gasto: " + str(timedelta(seconds=int(round(time_dif)))))

### Função Para Plot dos Erros

In [0]:
def plot_example_errors(cls_pred, correct):
    incorrect = (correct == False)

    images = images_test[incorrect]

    cls_pred = cls_pred[incorrect]

    cls_true = cls_test[incorrect]
    
    plot_images(images=images[0:9],
                cls_true=cls_true[0:9],
                cls_pred=cls_pred[0:9])

### Função Para o Plot da Confusion Matrix

In [0]:
def plot_confusion_matrix(cls_pred):

    cm = confusion_matrix(y_true=cls_test, y_pred=cls_pred)  

    for i in range(num_classes):
        class_name = "({}) {}".format(i, class_names[i])
        print(cm[i, :], class_name)

    class_numbers = [" ({0})".format(i) for i in range(num_classes)]
    print("".join(class_numbers))

### Função Para Calcular as Classificações

In [0]:
batch_size = 256

def predict_cls(images, labels, cls_true):
    num_images = len(images)

    cls_pred = np.zeros(shape=num_images, dtype=np.int)

    i = 0

    while i < num_images:
        j = min(i + batch_size, num_images)

        feed_dict = {x: images[i:j, :], y_true: labels[i:j, :]}

        cls_pred[i:j] = session.run(y_pred_cls, feed_dict=feed_dict)

        i = j

    correct = (cls_true == cls_pred)

    return correct, cls_pred

In [0]:
def predict_cls_test():
    return predict_cls(images = images_test,
                       labels = labels_test,
                       cls_true = cls_test)

### Função Para Acurácia de Classificação

In [0]:
def classification_accuracy(correct):
    return correct.mean(), correct.sum()

### Função Para Visualizar a Performance

In [0]:
def print_test_accuracy(show_example_errors=False, show_confusion_matrix=False):

    correct, cls_pred = predict_cls_test()
    
    acc, num_correct = classification_accuracy(correct)
    
    num_images = len(correct)

    msg = "Acurácia no Dataset de Teste: {0:.1%} ({1} / {2})"
    print(msg.format(acc, num_correct, num_images))

    if show_example_errors:
        print("Errors:")
        plot_example_errors(cls_pred=cls_pred, correct=correct)

    if show_confusion_matrix:
        print("Confusion Matrix:")
        plot_confusion_matrix(cls_pred=cls_pred)

### Função Para Plot dos Pesos Convolucionais

In [0]:
def plot_conv_weights(weights, input_channel=0):
    w = session.run(weights)

    print("Min:  {0:.5f}, Max:   {1:.5f}".format(w.min(), w.max()))
    print("Mean: {0:.5f}, Stdev: {1:.5f}".format(w.mean(), w.std()))
    
    w_min = np.min(w)
    w_max = np.max(w)
    abs_max = max(abs(w_min), abs(w_max))

    num_filters = w.shape[3]

    num_grids = math.ceil(math.sqrt(num_filters))
    
    fig, axes = plt.subplots(num_grids, num_grids)

    for i, ax in enumerate(axes.flat):
        if i < num_filters:
            img = w[:, :, input_channel, i]

            ax.imshow(img, vmin=-abs_max, vmax=abs_max, interpolation='nearest', cmap='seismic')
        
        ax.set_xticks([])
        ax.set_yticks([])
    
    plt.show()

### Função Para o Plot das Saídas das Camadas Convolucionais

In [0]:
def plot_layer_output(layer_output, image):
    feed_dict = {x: [image]}
    
    values = session.run(layer_output, feed_dict=feed_dict)

    values_min = np.min(values)
    values_max = np.max(values)

    num_images = values.shape[3]

    num_grids = math.ceil(math.sqrt(num_images))

    fig, axes = plt.subplots(num_grids, num_grids)

    for i, ax in enumerate(axes.flat):
        if i < num_images:
            img = values[0, :, :, i]

            ax.imshow(img, vmin=values_min, vmax=values_max, interpolation='nearest', cmap='binary')
        
        ax.set_xticks([])
        ax.set_yticks([])
    
    plt.show()

## Exemplos de Imagens de Input

In [0]:
def plot_distorted_image(image, cls_true):
    image_duplicates = np.repeat(image[np.newaxis, :, :, :], 9, axis=0)

    feed_dict = {x: image_duplicates}
    
    result = session.run(distorted_images, feed_dict=feed_dict)

    plot_images(images=result, cls_true=np.repeat(cls_true, 9))

In [0]:
def get_test_image(i):
    return images_test[i, :, :, :], cls_test[i]

In [0]:
img, cls = get_test_image(16)

In [0]:
plot_distorted_image(img, cls)

## Otimização

Dependendo do número de otimizações, isso pode levar de 1 a 15 horas.

In [0]:
num_inter = 120000
num_inter = 70000
optimize(num_iterations = num_inter)

## Resultados

Após 120.000 iterações de otimização, a precisão da classificação é de cerca de 79-80% no conjunto de teste. Exemplos de classificações erradas são plotados abaixo. Alguns são difíceis de reconhecer, mesmo para os seres humanos e outros são erros razoáveis, p. Ex. Entre um grande carro e um caminhão, ou entre um gato e um cachorro, enquanto outros erros parecem um pouco estranhos.

In [0]:
print_test_accuracy(show_example_errors=True, show_confusion_matrix=True)

### Pesos Convolucionais

O seguinte mostra alguns dos pesos (ou filtros) para a primeira camada convolucional. Existem 3 canais de entrada, portanto, há 3 desses conjuntos, que você pode plotar, alterando o `input_channel`.

Observe que os pesos positivos são vermelhos e os pesos negativos são azuis.

In [0]:
plot_conv_weights(weights=weights_conv1, input_channel=0)

In [0]:
plot_conv_weights(weights=weights_conv2, input_channel=1)

### Output das Camadas de Convolução

In [0]:
def plot_image(image):
    fig, axes = plt.subplots(1, 2)

    ax0 = axes.flat[0]
    ax1 = axes.flat[1]

    ax0.imshow(image, interpolation='nearest')
    ax1.imshow(image, interpolation='spline16')

    ax0.set_xlabel('Raw')
    ax1.set_xlabel('Smooth')
    
    plt.show()

In [0]:
img, cls = get_test_image(16)
plot_image(img)

In [0]:
plot_layer_output(output_conv1, image=img)

In [0]:
plot_layer_output(output_conv2, image=img)

### Labels de Classe Previstos

In [0]:
label_pred, cls_pred = session.run([y_pred, y_pred_cls], feed_dict={x: [img]})

In [0]:
np.set_printoptions(precision=3, suppress=True)

print(label_pred[0])

In [0]:
class_names[3]

In [0]:
class_names[5]

## Encerrando a Sessão

In [0]:
#session.close()

## Conclusão

Este jupyter notebook mostrou como construir uma Rede Neural convolutiva para classificar imagens no conjunto de dados CIFAR-10. A precisão da classificação foi de cerca de 79-80% no conjunto de teste.